In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Object, create a model to predict points scored

In [ ]:
df = pd.read_csv('../input/beginner-datasets/beginner_datasets/nba.csv')

In [ ]:
#observe the data
df.head()

In [ ]:
#Check that the data is clean
df.info()

In [ ]:
#data is mostly clean, some missing values in 3P%

In [ ]:
#visualize the data, as we are trying to predict points scored, let's check the
#relations between points scored and some of the variables we think might have the most 
#effect on this: MIN, FGM, FG%, 3PM, 3P%, FTM, FT%

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.scatter(x = 'MIN', y = 'PTS', data = df )

In [ ]:
plt.scatter(x = 'FGM', y = 'PTS', data = df)

In [ ]:
plt.scatter(x = 'FG%', y = 'PTS', data = df)

In [ ]:
plt.scatter(x = '3P Made', y = 'PTS', data = df)

In [ ]:
plt.scatter(x = '3P%', y = 'PTS', data = df)

In [ ]:
plt.scatter(x = 'FTM', y = 'PTS', data = df)

In [ ]:
plt.scatter(x = 'FT%', y = 'PTS', data = df)

In [ ]:
#After plotting the data we can note a mostly positive linear relationship between PTS and MIN as would 
#be expected, the more minutes you play the more likely you are to score more points.
#We note a near perfectly correlated linear relationship between PTS AND FGM
#A weaker positive association is noted between PTS and FG%, as players can score a low number of 
#points at a high field goal percentage
#The association between PTS and 3P Made and 3P% is weak as players can score a high number of points
#without shooting any three point shots
#FTM and PTS have a strong positive linear relationship
#FT% and PTS have a weaker positive relationship, and percentages are clustered around the 40%-80% range
#for low and high point scorers

In [ ]:
#For our MLE model we will use the variables FGM, FTM and 3P Made to predict PTS as 
#these FGM and FTM have a strong positive linear correlation, and 3P Made contribute to the
#points total despite have a weaker relationship on the scatterplot.
#Theoritically a points scored model would be points = 2*FGM + 1*FTM + 3*3P Made
#Let's see if we can replicate this with our practical model from the data

In [ ]:
#import linear model package
from sklearn import linear_model

In [ ]:
#build the linear model
points = linear_model.LinearRegression()
points.fit(df[['FGM','FTM','3P Made']], df.PTS)

In [ ]:
#Model Evalution
# print the intercept
print(points.intercept_)

In [ ]:
print(points.coef_)

In [ ]:
#Interpreting the coefficients
#Holding all other features fixed, a 1 unit increase in FGM
#is associated with a PTS increase of 2
#Holding all other features fixed, a 1 unit increase in FTM
#is associted with a PTS increase of 1
#Holding all other features fixed, a 1 unit increase in 3P Made
#is associated with a PTS increase of 0.99 - This needs to be scaled as it doesn't make
#sense, a 3P Made should be a associated with a points increase of 3

In [ ]:
#scale the 3P Made column
df['3P Made'] = 3*df['3P Made']

In [ ]:
#check to see if it worked
df.head()

In [ ]:
#scaling worked, rebuild the model
newpoints = linear_model.LinearRegression()
newpoints.fit = (df[['FGM','FTM','3P Made']],df.PTS)

In [ ]:
#Train Test Split
#Now let's split the data into a training set and a testing set. 
#We will train out model on the training set and then use the test set to evaluate the model.

In [ ]:
X = df[['FGM','FTM','3P Made']]
y = df['PTS']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, X, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [ ]:
#Predictions from our model
#Let's grab predictions off of our test set and see how the model performs

In [ ]:
pred = points.predict(X_test)

In [ ]:
plt.scatter(y_test, pred)

In [ ]:
import seaborn as sns

In [ ]:
sns.displot((y_test - pred), bins=50);

In [ ]:
test_pred = points.predict(X_test)
train_pred = points.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

In [ ]:
#R^2 is 0.96, model is accurate.